In [301]:
import numpy as np
import pandas as pd
from time import time

data = pd.read_csv("199541990_T_ONTIME/199541990_T_ONTIME_2015_3.csv")
airlines = pd.read_csv("L_UNIQUE_CARRIERS.csv-")
airports = pd.read_csv("1059086330_T_MASTER_CORD.csv")
airports = airports[(airports.AIRPORT_IS_LATEST == 1) & (airports.AIRPORT_IS_CLOSED == 0)]
wacs = pd.read_csv("L_WORLD_AREA_CODES.csv-")

start = time()
# Some airlines are solely subsidiaries, replace them with their parent airlines according to how 538 classifies them
#   http://fivethirtyeight.com/features/how-we-found-the-fastest-flights/#fn-5
# Additionally American Airlines and US Airways are merging, AirTran and Southwest have combined

# Expects data[['UNIQUE_CARRIER', 'FL_NUM']] as an argument
# Returns the new data['UNIQUE_CARRIER']
def renameAirline(airplane): 
    # Express Jet
    if airplane['UNIQUE_CARRIER'] == "EV":
        if airplane['FL_NUM'] >= 2500 and airplane['FL_NUM'] <= 2574: 
            return "AA"
        elif (airplane['FL_NUM'] >= 3255 and airplane['FL_NUM'] <= 4868
              or airplane['FL_NUM'] >= 5660 and airplane['FL_NUM'] <= 6189):
            return "UA"
        elif airplane['FL_NUM'] >= 4869 and airplane['FL_NUM'] <= 5632:
            return "DL"

    # Skywest Air
    elif airplane['UNIQUE_CARRIER'] == "OO":
        if (airplane['FL_NUM'] >= 2575 and airplane['FL_NUM'] <= 2649
            or airplane['FL_NUM'] >= 2901 and airplane['FL_NUM'] <= 2974
            or airplane['FL_NUM'] >= 6550 and airplane['FL_NUM'] <= 6629):
            return "AA"
        elif airplane['FL_NUM'] >= 3448 and airplane['FL_NUM'] <= 3499:
            return "AS"
        elif (airplane['FL_NUM'] >= 4438 and airplane['FL_NUM'] <= 4859
              or airplane['FL_NUM'] >= 7362 and airplane['FL_NUM'] <= 7439):
            return "DL"
        elif airplane['FL_NUM'] >= 4965 and airplane['FL_NUM'] <= 6539:
            return "UA"

    # Envoy Air
    elif airplane['UNIQUE_CARRIER'] == "MQ":
        return "AA"

    # US Airways
    elif airplane['UNIQUE_CARRIER'] == "US":
        return "AA"
    
    # AirTran Airways
    elif airplane['UNIQUE_CARRIER'] == "FL":
        return "WN"

    else:
        return airplane['UNIQUE_CARRIER']
    
data['UNIQUE_CARRIER'] = data[['UNIQUE_CARRIER', 'FL_NUM']].apply(axis=1, func=renameAirline);
stop = time()
print ("Done replacing airlines. Took {0:.2f} seconds".format(stop - start))

Done replacing airlines. Took 17.53 seconds


In [305]:
# Summarize data down to flights per airport per carrier so that the visualization doesn't have to compute the sum 
#   every time it's run
key_data = data[['UNIQUE_CARRIER', 'ORIGIN', 'ORIGIN_WAC', 'DEST', 'DEST_WAC', 'FLIGHTS']]
key_data = key_data.groupby(['UNIQUE_CARRIER', 'ORIGIN', 'ORIGIN_WAC', 'DEST', 'DEST_WAC'], as_index=False).sum()

# Filter out flight records involving a non-US airport to allow for easy visualization using D3's Albers Usa Projection
us_wacs = wacs[wacs.Code.isin([1, 2, 11, 12, 13, 14, 15, 16, 21, 22, 23, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 
                               43, 44, 45, 51, 52, 53, 54, 61, 62, 63, 64, 65, 66, 67, 71, 72, 73, 74, 81, 82, 83, 
                               84, 85, 86, 87, 88, 91, 92, 93])]
us_data = key_data[key_data.ORIGIN_WAC.isin(us_wacs.Code) & key_data.DEST_WAC.isin(us_wacs.Code)]

# Biggest airlines
airline_flights = us_data[['UNIQUE_CARRIER', 'FLIGHTS']].groupby(['UNIQUE_CARRIER'], as_index=False).sum()
airline_flights.sort('FLIGHTS', ascending=False, inplace=True)
airline_flights = airline_flights.merge(airlines, left_on="UNIQUE_CARRIER", right_on="Code")

# Append airline name so that the visualization doesn't have to do the lookup every time it's run
us_data = us_data.merge(airlines, left_on="UNIQUE_CARRIER", right_on="Code")
us_data.drop("Code", axis=1, inplace=True)
us_data.rename(columns={"Description": "AIRLINE"}, inplace=True)


# Filter out flight records involving a non-US airport to allow for easy visualization using D3's Albers Usa Projection
us_airports = airports[airports.AIRPORT_WAC.isin(us_wacs.Code)]

# Shorten airports to relevant fields for the visualization
us_airports = us_airports[["AIRPORT", "LATITUDE", "LONGITUDE", "DISPLAY_AIRPORT_NAME"]]

us_data.head()
#us_airports.head()
airline_flights
us_data[(us_data.ORIGIN=="ORD") & (us_data.DEST=="ATL")].head()
#key_airlines

,AIRPORT_SEQ_ID,AIRPORT_ID,AIRPORT,DISPLAY_AIRPORT_NAME,DISPLAY_AIRPORT_CITY_NAME_FULL,AIRPORT_WAC,AIRPORT_COUNTRY_NAME,AIRPORT_COUNTRY_CODE_ISO,AIRPORT_STATE_NAME,AIRPORT_STATE_CODE,...,LON_DEGREES,LON_HEMISPHERE,LON_MINUTES,LON_SECONDS,LONGITUDE,AIRPORT_START_DATE,AIRPORT_THRU_DATE,AIRPORT_IS_CLOSED,AIRPORT_IS_LATEST,Unnamed: 28
0,1000101,10001,01A,Afognak Lake Airport,"Afognak Lake, AK",1,United States,US,Alaska,AK,...,152,W,54,24,-152.906667,2007-07-01,NaN,0,1,NaN
1,1000301,10003,03A,Bear Creek Mining Strip,"Granite Mountain, AK",1,United States,US,Alaska,AK,...,161,W,4,18,-161.071667,2007-07-01,NaN,0,1,NaN
2,1000401,10004,04A,Lik Mining Camp,"Lik, AK",1,United States,US,Alaska,AK,...,163,W,10,0,-163.166667,2007-07-01,NaN,0,1,NaN
3,1000501,10005,05A,Little Squaw Airport,"Little Squaw, AK",1,United States,US,Alaska,AK,...,148,W,11,2,-148.183889,2007-08-01,NaN,0,1,NaN
4,1000601,10006,06A,Kizhuyak Bay,"Kizhuyak, AK",1,United States,US,Alaska,AK,...,152,W,52,58,-152.882778,2007-10-01,NaN,0,1,NaN


In [231]:
data.columns.values

array(['FL_DATE', 'UNIQUE_CARRIER', 'TAIL_NUM', 'FL_NUM', 'ORIGIN',
       'ORIGIN_WAC', 'DEST', 'DEST_WAC', 'CANCELLED', 'CANCELLATION_CODE',
       'DIVERTED', 'ACTUAL_ELAPSED_TIME', 'FLIGHTS', 'CARRIER_DELAY',
       'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY',
       'LATE_AIRCRAFT_DELAY', 'DIV_AIRPORT_LANDINGS', 'DIV_REACHED_DEST',
       'DIV_ACTUAL_ELAPSED_TIME', 'Unnamed: 21'], dtype=object)

In [303]:
us_data.to_csv("us_flights.csv")
#us_data[us_data.ORIGIN=="ORD"].to_csv("us_flights.csv")
us_airports.to_csv("us_airports.csv")
key_airlines.to_csv("key_airlines.csv")

In [245]:
data[data.UNIQUE_CARRIER == 'US'].head()

,FL_DATE,UNIQUE_CARRIER,TAIL_NUM,FL_NUM,ORIGIN,ORIGIN_WAC,DEST,DEST_WAC,CANCELLED,CANCELLATION_CODE,...,FLIGHTS,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,DIV_AIRPORT_LANDINGS,DIV_REACHED_DEST,DIV_ACTUAL_ELAPSED_TIME,Unnamed: 21
10467,2014-07-01,US,N602AW,403,DFW,74,PHX,81,0,NaN,...,1,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
10468,2014-07-01,US,N602AW,403,PHX,81,SFO,91,0,NaN,...,1,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
10469,2014-07-01,US,N602AW,403,SFO,91,PHL,23,0,NaN,...,1,0,0,24,0,0,0,NaN,NaN,NaN
10470,2014-07-01,US,N825AW,406,DTW,43,PHX,81,0,NaN,...,1,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
10471,2014-07-01,US,N825AW,406,PHX,81,DTW,43,0,NaN,...,1,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN


520875    WN
520876    WN
520877    WN
520878    WN
520879    WN
dtype: object